In [1]:
import pandas as pd
from datetime import datetime
import glob
import os


# 엑셀 파일들이 저장된 폴더 경로
folder_path = r'C:\Users\82103\Desktop\YONSEI\2024-1\OR확정모델\프로젝트\dividends'  

# 폴더 내의 모든 CSV 파일을 찾기

csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# 파일 이름을 키로, 데이터프레임을 값으로 하는 딕셔너리 생성
dict = {}

for file in csv_files:
    try:
        # 파일 이름에서 확장자 제거하여 변수 이름으로 사용
        file_name = os.path.splitext(os.path.basename(file))[0]
        df = pd.read_csv(file)
        dict[file_name] = df
    except Exception as e:
        print(f"Error reading {file}: {e}")

자료를 정리할 DF 생성

In [2]:
date_range = pd.date_range(start='2019-01', end='2024-04', freq='M')
date_strings = date_range.strftime('%Y-%m')
df = pd.DataFrame(date_strings, columns=['Date'])
df

,Date
0,2019-01
1,2019-02
2,2019-03
3,2019-04
4,2019-05
...,...
58,2023-11
59,2023-12
60,2024-01
61,2024-02


In [10]:
for key in dict:
    a = dict[key]
    a = a.rename(columns = {'Dividends' : key.replace("_d", "")})
    
    a["Date"] = pd.to_datetime(a["Date"])
    a["Date"] = a["Date"].dt.strftime('%Y-%m')
   
    date_list = []
    for i in a["Date"]:
        date_str = i
        date_obj = datetime.strptime(date_str, "%Y-%m")
        if datetime(date_obj.year, date_obj.month, 1) >= datetime(2019, 1, 1):
            date_list.append(i)

    filtered_df = a[a['Date'].isin(date_list)]
    
    merged_df = pd.merge(df, filtered_df, on = "Date", how = "left")
    merged_df.fillna(0, inplace = True)
    df = merged_df
    
print(df)

       Date   AFL    ALB   AOS   APD    ATO    BFB   BRO  CHRW  CINF  ...  \
0   2019-01  0.00  0.000  0.22  0.00  0.000  0.000  0.00  0.00  0.00  ...   
1   2019-02  0.27  0.000  0.00  0.00  0.525  0.000  0.08  0.50  0.00  ...   
2   2019-03  0.00  0.368  0.00  1.16  0.000  0.166  0.00  0.00  0.56  ...   
3   2019-04  0.00  0.000  0.22  0.00  0.000  0.000  0.00  0.00  0.00  ...   
4   2019-05  0.27  0.000  0.00  0.00  0.525  0.000  0.08  0.00  0.00  ...   
..      ...   ...    ...   ...   ...    ...    ...   ...   ...   ...  ...   
63  2023-11  0.42  0.000  0.00  0.00  0.805  0.218  0.00  0.61  0.00  ...   
64  2023-12  0.00  0.400  0.00  1.75  0.000  0.000  0.00  0.00  0.75  ...   
65  2024-01  0.00  0.000  0.32  0.00  0.000  0.000  0.00  0.00  0.00  ...   
66  2024-02  0.50  0.000  0.00  0.00  0.805  0.000  0.13  0.61  0.00  ...   
67  2024-03  0.00  0.400  0.00  1.77  0.000  0.218  0.00  0.00  0.81  ...   

     PPG    ROP  SPGI   SWK   SYY   TGT  TROW       WMT   WST   XOM  
0   0

파일에 저장하기

In [11]:
df.to_csv('dividends_찐.csv', index=False)